<a href="https://colab.research.google.com/github/Lilianllland/JPM-Simulation/blob/main/Nat_Gas_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [143]:
import pandas as pd
import numpy as np


In [ ]:
df = pd.read_csv('/content/Nat_Gas.csv',index_col= 'Dates', parse_dates=True) #make sure pandas get right date format
df= df.dropna()
print('Shape of data', df.shape)
df.head()

In [ ]:
df['Prices'].plot(figsize=(12,5))

Check for Stationary

In [206]:
from statsmodels.tsa.stattools import adfuller

def ad_test(dataset):
  dftest = adfuller(dataset, autolag = 'AIC')
  print("1. ADF : ",dftest[0])
  print("2. P-Value : ", dftest[1])
  print("3. Num Of Lags : ", dftest[2])
  print("4. Num Of Observations Used For ADF Regression and Critical Values Calculation :", dftest[3])
  print("5. Critical Values :")
  for key, val in dftest[4].items():
      print("\t",key, ": ", val)

In [207]:
ad_test(df['Prices'])

1. ADF :  0.21807686169999427
2. P-Value :  0.973257438844869
3. Num Of Lags :  10
4. Num Of Observations Used For ADF Regression and Critical Values Calculation : 37
5. Critical Values :
	 1% :  -3.6209175221605827
	 5% :  -2.9435394610388332
	 10% :  -2.6104002410518627


P-Value :  0.973257438844869 >= 0.05, which accepts our hypothesis that the Price is non-stationary.

The natural gas prices is **non-stationary**. It has ***Trends*** and ***Seasonality***.

Then, let's figure out orders for ARIMA Model

In [ ]:
!pip install pmdarima

In [209]:
from pmdarima import auto_arima
import warnings
warnings.filterwarnings("ignore")

In [ ]:
stepwise_fit = auto_arima(df['Prices'], trace=True,
                          suppress_warnings=True)

stepwise_fit.summary()

*  Best model always be the smallest AIC value ARIMA

*  SARIMA is ARIMA with seasonality.

Differencing

In [211]:
df['Prices First Difference'] = df['Prices'] - df['Prices'].shift(1)

In [ ]:
df['Prices'].shift(1)

In [222]:
df['Seasonal First Difference'] = df['Prices'] - df['Prices'].shift(12) #instead of 1, we use 12 to predict by season.

In [ ]:
df.tail(14)

In [221]:
from statsmodels.tsa.stattools import adfuller

In [29]:
test_result=adfuller(df['Prices'])

In [219]:
def adfuller_test(prices):
    result=adfuller(prices)
    labels = ['ADF Test Statistic','p-value','#Lags Used','Number of Observations Used']
    for value,label in zip(result,labels):
        print(label+' : '+str(value) )
    if result[1] <= 0.05:
        print("strong evidence against the null hypothesis(Ho), reject the null hypothesis. Data has no unit root and is stationary")
    else:
        print("weak evidence against null hypothesis, time series has a unit root, indicating it is non-stationary ")


In [ ]:
adfuller_test(df['Seasonal First Difference'].dropna())

In [ ]:
df['Seasonal First Difference'].plot()

In [226]:
import matplotlib.pyplot as plt

In [227]:
import statsmodels.api as sm

In [228]:
from statsmodels.tsa.arima.model import ARIMA

In [230]:
model=ARIMA(df['Prices'],order=(1,1,1))
model_fit=model.fit()

In [ ]:
model_fit.summary()

In [281]:
model=sm.tsa.statespace.SARIMAX(df['Prices'],order=(1,1,1),seasonal_order=(1,1,1,12))
results=model.fit()

In [ ]:
df['forecast']=results.predict(start=30,end=48,dynamic=True)
df[['Prices','forecast']].plot(figsize=(12,8))

In [ ]:
df.count()

In [277]:
from pandas.tseries.offsets import DateOffset
future_dates=[df.index[-1]+ DateOffset(months=x)for x in range(0,24)]

In [278]:
future_datest_df=pd.DataFrame(index=future_dates[1:],columns=df.columns)

In [279]:
future_df=pd.concat([df,future_datest_df])

In [ ]:
future_df['forecast'] = results.predict(start =49, end = 63, dynamic=True)
future_df[['Prices', 'forecast']].plot(figsize=(12, 8))